In [19]:
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from scipy.io.arff import loadarff

In [46]:
names = ['user', 'activity', 'timestamp', 'x-accel', 'y-accel', 'z-accel', "NaN"]

df = loadarff('../../dataset/WISDM_ar_v1.1_transformed.arff')

df[1]

Dataset: activity_recognition_labeled
	"UNIQUE_ID"'s type is numeric
	"user"'s type is nominal, range is ('33', '17', '29', '13', '20', '27', '6', '15', '32', '36', '18', '35', '11', '16', '5', '10', '28', '26', '14', '24', '12', '23', '4', '30', '34', '8', '31', '21', '3', '22', '1', '25', '9', '2', '7', '19')
	"X0"'s type is numeric
	"X1"'s type is numeric
	"X2"'s type is numeric
	"X3"'s type is numeric
	"X4"'s type is numeric
	"X5"'s type is numeric
	"X6"'s type is numeric
	"X7"'s type is numeric
	"X8"'s type is numeric
	"X9"'s type is numeric
	"Y0"'s type is numeric
	"Y1"'s type is numeric
	"Y2"'s type is numeric
	"Y3"'s type is numeric
	"Y4"'s type is numeric
	"Y5"'s type is numeric
	"Y6"'s type is numeric
	"Y7"'s type is numeric
	"Y8"'s type is numeric
	"Y9"'s type is numeric
	"Z0"'s type is numeric
	"Z1"'s type is numeric
	"Z2"'s type is numeric
	"Z3"'s type is numeric
	"Z4"'s type is numeric
	"Z5"'s type is numeric
	"Z6"'s type is numeric
	"Z7"'s type is numeric
	"Z8"'s type is

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X = df_clear.iloc[:, 3:6]
y = df_clear['activity']

X.shape, y.shape

((1098204, 3), (1098204,))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)

In [6]:
col = X_train.columns
col

Index(['x-accel', 'y-accel', 'z-accel'], dtype='object')

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

scaled_X_train = pd.DataFrame(data = X_train, columns = col)
scaled_X_test = pd.DataFrame(data = X_test, columns = col)
scaled_X_train['label'] = y_train.values
scaled_X_test['label'] = y_test.values

X_train = scaled_X_train
X_test = scaled_X_test

scaled_X_train.head(10)

,x-accel,y-accel,z-accel,label
0,2.220233,1.048787,-1.086362,Jogging
1,0.310554,0.241872,0.484817,Downstairs
2,-0.091561,0.043110,-0.362306,Downstairs
3,0.557672,-0.446379,0.152492,Jogging
4,-0.186606,0.384268,0.148286,Upstairs
5,-0.202690,-1.315593,1.904557,Sitting
6,0.385128,1.753354,1.267761,Jogging
7,-0.047693,-0.434513,-0.580030,Jogging
8,0.401213,-1.042666,1.050037,Jogging
9,2.473200,0.054976,2.499796,Jogging


In [8]:
# The number of steps within one time segment
TIME_PERIODS = 80

# The steps to take from one segment to the next; if this value is equal to TIME_PERIODS, then there is
# no overlap between the segments
STEP_DISTANCE = 40

In [9]:
unique, counts = np.unique(y_train, return_counts=True)
print ("Train data label statistics::")
print (np.asarray((unique, counts)).T)  

unique, counts = np.unique(y_test, return_counts=True)
print ("Test data label statistics::")
print (np.asarray((unique, counts)).T) 

Train data label statistics::
[['Downstairs' 67320]
 ['Jogging' 228866]
 ['Sitting' 40229]
 ['Standing' 32391]
 ['Upstairs' 82298]
 ['Walking' 284692]]
Test data label statistics::
[['Downstairs' 33107]
 ['Jogging' 113310]
 ['Sitting' 19710]
 ['Standing' 16004]
 ['Upstairs' 40571]
 ['Walking' 139706]]


In [10]:
from scipy import stats

def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

TIME_STEPS = 1
STEP = 1

X_train, y_train = create_dataset(X_train[col], X_train.label, TIME_STEPS,
                                  STEP)
X_test, y_test = create_dataset(X_test[col], X_test.label, TIME_STEPS, STEP)

print(X_train.shape, y_train.shape)

(735795, 1, 3) (735795, 1)


In [11]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown = "ignore", sparse = False)
enc = enc.fit(y_train)

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

/Users/kimtaeyoon/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# cnn model vary kernel size
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.layers import TimeDistributed, Conv1D, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

In [13]:
verbose,epochs,batch_size=1,10,32 

n_timesteps,n_features,n_outputs=X_train.shape[1],X_train.shape[2],y_train.shape[1]

In [14]:
# model = keras.Sequential()
# model.add(Bidirectional(LSTM(units = 128, input_shape = [X_train.shape[1], X_train.shape[2]])))
# model.add(Dropout(rate = 0.5))
# model.add(Dense(units = 128, activation = "relu"))
# model.add(Dense(y_train.shape[1], activation = "softmax"))
# model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["acc"])

In [15]:
model = Sequential([
    LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')
])

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                4608      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 128)               4224      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 6)                 390       
                                                                 
Total params: 17,478
Trainable params: 17,478
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(X_train, y_train, epochs = 10, batch_size = 24, validation_split = 0.33, shuffle = True)

Epoch 1/10
20541/20541 [==============================] - 33s 2ms/step - loss: 1.0890 - acc: 0.5829 - val_loss: 1.0626 - val_acc: 0.5824
Epoch 2/10
20541/20541 [==============================] - 32s 2ms/step - loss: 1.0361 - acc: 0.6014 - val_loss: 1.0804 - val_acc: 0.5762
Epoch 3/10
20541/20541 [==============================] - 31s 2ms/step - loss: 1.0218 - acc: 0.6063 - val_loss: 1.1106 - val_acc: 0.5700
Epoch 4/10
20541/20541 [==============================] - 31s 2ms/step - loss: 1.0122 - acc: 0.6105 - val_loss: 1.1089 - val_acc: 0.5606
Epoch 5/10
20541/20541 [==============================] - 32s 2ms/step - loss: 1.0065 - acc: 0.6124 - val_loss: 1.1391 - val_acc: 0.5665
Epoch 6/10
20541/20541 [==============================] - 32s 2ms/step - loss: 1.0025 - acc: 0.6140 - val_loss: 1.1387 - val_acc: 0.5396
Epoch 7/10
 9470/20541 [============>.................] - ETA: 14s - loss: 1.0009 - acc: 0.6155

KeyboardInterrupt: 

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns

acc = history.history['acc']
val_acc = history.history['val_acc']

x = np.arange(0, 6)
plt.plot(x, acc, 'y', label='Training acc')
plt.plot(x, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [ ]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

y_test_arg=np.argmax(y_test,axis=1)
Y_pred = np.argmax(model.predict(X_test),axis=1)
print('Confusion Matrix')
print(confusion_matrix(y_test_arg, Y_pred))

In [ ]:
score = model.evaluate(X_test, y_test)

pd.DataFrame(score)